# Intuitive Value Metric Measurment

price to earnings ratio

price to book ratio

price to sales ratio

Enterprice earnings divided by earnings before interest, taxes , depreciation, amorization

Enterprice earnings divided by Gross Profit

In [ ]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math
import statistics
import pprint

# Importing data

Importing SMP-500 

importing api token

In [ ]:
# SMP 500 CSV
stocks = pd.read_csv("sp_500_stocks.csv")

# api
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

stocks


# TESTING METRICS

please keep this code commented... just here to show variables used for value metric calculation

In [ ]:
# symbol = "AAPL"
# batch_api_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
# data = requests.get( batch_api_call  ).json()

# # BULK DATA
# # pprint.pprint(data)

# # Price to earnings ratio
# pe_ratio = data[symbol]['quote']['peRatio'] 

# # price to book ratio
# # NOTE - P/B ratios under 1 are typically considered solid investments.
# pb_ratio = data[symbol]['advanced-stats']['priceToBook'] 

# # price to sales ratio
# ps_ratio = data[symbol]['advanced-stats']['priceToSales'] 

# # Enterprice earnings divided by earnings before interest, taxes , depreciation, amorization
# enterprise_value = data[symbol]['advanced-stats']['enterpriseValue'] 
# EBITA = data[symbol]['advanced-stats']['EBITDA'] 
# ev_to_ebita = enterprise_value / EBITA

# # Enterprice earnings divided by Gross Profit
# gross_profit = data[symbol]['advanced-stats']['grossProfit'] 
# ev_to_gross_profit = enterprise_value / gross_profit


# print(f'{pe_ratio = }')
# print(f'{pb_ratio = }')
# print(f'{ps_ratio = }\n')

# print(f'{enterprise_value = }')
# print(f'{EBITA = }')
# print(f'{ev_to_ebita = }\n')


# print(f'{gross_profit = }')
# print(f'{ev_to_gross_profit = }')

# SORTING ALL STOCKS INTO 2D ARRAY HOLDING A LIST OF 100 STOCKS EACH
[
    [100],
    [100],
    [100],
    [100],
    [100],
]

# TURN EACH LIST INTO A STRING 
[
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
    'X' * 100 ,
]

In [ ]:
# function to split bigger list into smaller lists
def chunks( lst , n  ):
    ''' yeild successive n-sized lists from lst'''
    for i in range( 0 , len(lst) , n ):
        yield lst[i: i + n]

# 2 dimentional list with each inner list containing 100 tickers
symbol_groups = list(chunks( stocks['Ticker'] , 100  ) )
symbol_strings = []

# turns each list into usable string for api call
ignored_stocks = []
for i in range( 0 , len(symbol_groups) , 1  ):
    try:
        joined_arr =  ",".join(symbol_groups[i])
        symbol_strings.append( joined_arr )
    except:
        ignored_stocks.append( symbol_groups[i] )

ignored_stocks

Creating final output dataframe

In [ ]:
# creating columns for dataframe
my_columns = [
    'Ticker',
    'Price',
    'Number of Shares To Buy',
    'Price to Earnings Ratio',
    'P/E Percentile',
    'Price to Book Ratio',
    'P/B Percentile',
    'Price to Sales Ratio',
    'P/S Percentile',
    "EV/EBITA",
    "EV/EBITA Percentile",
    "EV/GP",
    "EV/GP Percentile",
    "Robust Value Score"
]

final_dataframe = pd.DataFrame(columns = my_columns)

# final_dataframe

In [ ]:
# create url for each list of 100 stocks
# request data for the sub list
for group_string in symbol_strings:
    batch_api_call = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={group_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get( batch_api_call ).json()

    # arr of every ticker in sub list
    stock_array_in_sub_list = group_string.split(',')
    for symbol in stock_array_in_sub_list:
        symbol = symbol.upper()
        if symbol not in data:
            ignored_stocks.append(symbol)
            continue

        # fundementals
        price = data[symbol]['quote']['latestPrice'] 
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue'] 
        EBITA = data[symbol]['advanced-stats']['EBITDA'] 
        gross_profit = data[symbol]['advanced-stats']['grossProfit'] 

        # price ratios
        pe_ratio = data[symbol]['quote']['peRatio'] 
        pb_ratio = data[symbol]['advanced-stats']['priceToBook'] 
        ps_ratio = data[symbol]['advanced-stats']['priceToSales'] 
        
        # enterprise ratios (some EBITA datapoints come out as NONE-TYPE)
        
        try: # EBITA
            ev_to_ebita = enterprise_value / EBITA
        except TypeError:
            print(symbol, ": N/A - EBITA")
            ev_to_ebita = np.NaN
        
        try: # GROSS PROFITS
            ev_to_gross_profit = enterprise_value / gross_profit
        except TypeError:
            print(symbol, ": N/A - Gross Profit")
            ev_to_gross_profit = np.NaN


        #***** append to final_dataframe *****
        final_dataframe = final_dataframe.append(
            pd.Series(
                [
                    symbol,
                    price,
                    "N/A",
                    pe_ratio,
                    "N/A",
                    pb_ratio,
                    "N/A",
                    ps_ratio,
                    "N/A",
                    ev_to_ebita,
                    "N/A",
                    ev_to_gross_profit,
                    "N/A",
                    "N/A",
                ],
                index = my_columns 
            ),
            ignore_index = True #index will automatically be calculated for you on the pandas data frame
        )


# final_dataframe

DEALING WITH MISSING DATA FROM NONE-TYPES

In [ ]:
# finding missing data
# final_dataframe[ final_dataframe.isnull().any(axis=1) ]

In [ ]:
# MODIFY ALL NULL TYPES IN DATAFRAME INTO THE AVERAGE OF FOUND NUMBERS SUCH COLUMN
numerical_columns = [
    'Price to Earnings Ratio',
    'Price to Book Ratio',
    'Price to Sales Ratio',
    'EV/EBITA',
    'EV/GP'
]

for col in numerical_columns:
    final_dataframe.fillna(final_dataframe[col].mean() , inplace = True)




# check if there are any nonetypes
# NOTE dataframe should be empty (run code below to see missing data stocks)
# final_dataframe[ final_dataframe.isnull().any(axis=1) ]



# NOTE all NONE-TYPES are replaced with average of found numbers in that column
# final_dataframe

# CALCULATING VALUE PERCENTILES

    'P/E Percentile'
    'P/B Percentile'
    'P/S Percentile'
    "EV/EBITA Percentile"
    "EV/GP Percentile"

In [ ]:
metrics = {
    'Price to Earnings Ratio' : 'P/E Percentile',
    'Price to Book Ratio' : 'P/B Percentile',
    'Price to Sales Ratio' : 'P/S Percentile',
    "EV/EBITA" : "EV/EBITA Percentile",
    "EV/GP" : "EV/GP Percentile"
}

for metric in metrics.keys():
    for row in final_dataframe.index:
        # finding percentile of current row vs entire column
        final_dataframe.loc[ row , metrics[metric]  ] = stats.percentileofscore( 
            final_dataframe[ metric ] , #entire column
            final_dataframe.loc[ row  , metric  ] #comparison value
          ) / 100


# final_dataframe

# CALCULATING RV SCORE

robust value score - mean of percentile scores of value metrics

    'P/E Percentile'
    'P/B Percentile'
    'P/S Percentile'
    "EV/EBITA Percentile"
    "EV/GP Percentile"


In [ ]:

for row in final_dataframe.index:
    value_metrics = []
    for metric in metrics.keys():
        value_metrics.append(  final_dataframe.loc[  row ,  metrics[metric] ]  )
    final_dataframe.loc[ row  , 'Robust Value Score'  ] = statistics.mean( value_metrics )


# final_dataframe

# SELECTING 50 BEST STOCKS

In [ ]:
# Sorting values in DECENDING order and saving it for later use using INPLACE = TRUE
final_dataframe.sort_values( 'Robust Value Score' , ascending = True , inplace = True  )

# save final_dataframe[:50] as original 
final_dataframe = final_dataframe[:50]

# Resetting index of sorted data_frame
final_dataframe.reset_index(  inplace = True  )

# # removing Old Index Column
final_dataframe.drop(labels='index', axis= 'columns', inplace=True, errors='raise')

# final_dataframe

# CALCULATING AMOUNT OF SHARES TO BUY
    equal basket 

In [ ]:
# validate user input for portfollio size '$$$'
def user_portfollio_size_input(portfollio_size):
    try:
        val = float(portfollio_size)
        print(f"Your have entered: ${val}\n")
        return True
    except ValueError:
        print("ERROR: Please type a numerical value")
        return False

portfollio_size = input("Please enter value of your portfollio: $")
while not user_portfollio_size_input(portfollio_size):
    portfollio_size = input("Please enter value of your portfollio: $")

In [ ]:
# calculate position size
position_size = float(portfollio_size) / len(final_dataframe.index)
print(f"Your position size is ${'{:,.2f}'.format(position_size)}")

In [ ]:
# update final data frame
for x in range(0 , len(final_dataframe) , 1):
    final_dataframe.loc[x , "Number of Shares To Buy"] =  position_size /  final_dataframe.loc[x , "Price"]


final_dataframe

# CREATING EXCEL FILE

In [ ]:
# init writer objects
writer = pd.ExcelWriter('Robust_Value_Dataframe.xlsx' , engine='xlsxwriter')
final_dataframe.to_excel(writer , "Robust Value Metrics" , index = False)

# writer style init
font_color = "ffffff"
background_color = "#0a0a23"

# format style init
string_format = writer.book.add_format(
    {
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
dollar_format = writer.book.add_format(
    {
        'num_format' : "$0.00",
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
float_format = writer.book.add_format(
    {
        'num_format' : "0.00",
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)
percent_format = writer.book.add_format(
    {
        'num_format' : "0.00%",
        'font_color' : font_color,
        "bg_color" : background_color,
        "border" : 1
    }
)

# applying formats to .xlsx file
column_formats = {
    'A' : ['Ticker' , string_format],
    'B' : ['Price' , dollar_format],
    'C' : ['Number of Shares To Buy' , float_format],
    'D' : ['Price to Earnings Ratio' , float_format],
    'E' : ['P/E Percentile' , percent_format],
    'F' : ['Price to Book Ratio' , float_format],
    'G' : ['P/B Percentile' , percent_format],
    'I' : ['Price to Sales Ratio' , float_format],
    'J' : ['P/S Percentile' , percent_format],
    'K' : ['EV/EBITA' , float_format],
    'L' : ['EV/EBITA Percentile' , percent_format],
    'M' : ['EV/GP' , float_format],
    'N' : ['EV/GP Percentile' , percent_format],
    'O' : ['Robust Value Score' , percent_format],
}
# loop through formats and apply to excel sheet columns
for col in  column_formats.keys():

    # columns format
    writer.sheets["Robust Value Metrics"].set_column( f"{col}:{col}" , 15 , column_formats[col][1] )

    # headers format
    writer.sheets["Robust Value Metrics"].write( f"{col}1" , column_formats[col][0] , column_formats[col][1] )


# SAVE .xlsx
writer.save()



# SAVE INPUTS
with open("user_inputs.txt" , "w") as f:
    f.write( f"Amount Specified: ${float(portfollio_size)}\n"  )
    f.write( f"Amount Invested Into Each Stock: ${position_size}\n\n"  )
    f.write( "------ IGNORED STOCKS ------" )
    if len(ignored_stocks) > 0 :
        f.write("\n" + str(ignored_stocks)  )
    else:
        f.write("NONE")

In [ ]:
for row in final_dataframe.index:
    print(final_dataframe.loc[row , "Robust Value Score"])